In [ ]:
# import required packages
import boto3
import json

# aws service client
s3_obj = boto3.client('s3')

In [ ]:
# analyze image with Rekognition Custom Label
s3_bucket_name = "nutcha-sagemaker-bucket"

# get image keys from s3 bucket
response = s3_obj.list_objects_v2(
    Bucket=s3_bucket_name,
    MaxKeys=100,
    Prefix='images'
)

# provide keys as input
keys = []
for obj in response['Contents']:
    keys.append(obj['Key'])
print("Number of keys = {}".format(len(keys)))

In [ ]:
# import opencv for image processing
!pip3 install opencv-python &> /dev/null
import cv2
import numpy as np

target_s3_key = "target_images"

# loop in to image keys
for s3_key in keys:
    print("Working on '{}''".format(s3_key))
    try:
        # split prefix, image and suffix
        file_name = s3_key.split('/')[-1]
        first_name = file_name.split('.')[0]
        last_name = file_name.split('.')[1]

        # get image object from s3 bucket
        response = s3_obj.get_object(Bucket=s3_bucket_name,Key=s3_key)
        
        # read data from s3 as bytes
        image_raw = response['Body']
        image_bytes = image_raw.read()

        # convert image bytes array into numpy array
        nparr = np.frombuffer(image_bytes, np.uint8)
        img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        
        # make image as blur
        img_blur = cv2.medianBlur(img_np,5)
        # ret,th1 = cv2.threshold(img_blur,127,255,cv2.THRESH_BINARY)

        # convert byted image back to original image format
        success, encoded_image = cv2.imencode('.jpg', img_blur)
        print("Convert image status = {}".format(success))

        # put image to back to s3 bucket
        response = s3_obj.put_object(
            Body=encoded_image.tobytes(),
            Bucket=s3_bucket_name,
            Key=target_s3_key + "/" + first_name + "_new" + "." + last_name
        )
        print("Uploaded success")
        
    except IndexError as ie:
        # in case of key failure
        print(ie)

In [ ]:
# import required packages
from IPython.display import Image, display

# aws service client
s3_obj = boto3.client('s3')
s3_key = target_s3_key + '/'
s3_filename = "src_image_new.jpg"

# display image on output
display(Image(url=s3_obj.generate_presigned_url('get_object', Params={'Bucket': s3_bucket_name, 'Key': s3_key + s3_filename})))